In [2]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import itertools as it
import tabulate as tb
from functools import reduce
from operator import concat
from copy import deepcopy

blocks = []


def main(K, M, BC, Dir, Man, Nodes):
  ## K      - The number of source blocks to encode
  ## M      - The number of parity blocks produced
  ## BC     - Initial block count
  ## Dir    - Coding direction
  ## Man    - Manifest block layout
  ## Nodes  - Nodes count
  ##

  (rb, eg, n) = calcGeometry(K, M, BC)        # Calculate geometry
  blocks = list(range(1, rb + 1))             # Generate source blocks
  ecGroups = makeEcGroups(blocks, K, M, Dir)  # Calculate EC groups
  # manifestBlocks = makeManifestBlocks(blocks, K, M, Man)
  manifestBlocks = []

  if Man == "Append":
    if Dir == "Horizontal":
      manifestBlocks = reduce(
          concat,
          [g[:K] for g in ecGroups] + [g[-M:] for g in ecGroups])
    else:
      manifestBlocks = reduce(concat, zip(*ecGroups))
  else:
    manifestBlocks = reduce(concat, ecGroups)

  print("\n")
  print(K, M, BC, Dir, Man, Nodes)
  print("Rounded Blocks:", rb)
  print("Encoding Groups:", eg)
  print("K + M = ", n)
  print("Source Blocks:", blocks)
  print("Read Order:", manifestBlocks)
  print("\n",
        tb.tabulate(
            [[", ".join(map(str, g[:K])), " ,".join(map(str, g[-M:]))]
             for g in ecGroups],
            ["Source", "Parity"]))

  print("\n",
        tb.tabulate(list(map(list, mapNodes(manifestBlocks, Nodes).items())),
                    ["Nodes", "Blocks"]))


def calcGeometry(K, M, OB):
  RB = OB + (K - (OB % K))
  EG = (RB // K)
  N = K + M

  return (RB, EG, N)


def makeManifestBlocks(
        blocks,
        k, m,
        layout):

  offset = len(blocks)
  res = []

  steps = len(blocks) // (len(blocks) // k)
  if layout == "Inline":
    for b in range(0, len(blocks), steps):
      res += blocks[b:b + steps] + list(range(offset + 1, (offset + 1 + m)))
      offset += m
  else:
    res = deepcopy(blocks)
    for b in range(0, len(blocks), steps):
      res += list(range(offset + 1, (offset + 1 + m)))
      offset += m

  return res


def makeEcGroups(
        blocks,
        k, m,
        direction):

  offset = len(blocks)
  res = []

  groups = (len(blocks) // k)
  steps = len(blocks) // groups
  if direction == "Horizontal":
    for b in range(0, len(blocks), steps):
      res.append(
          blocks[b:b + steps] +
          list(range(offset + 1, (offset + 1 + m))))
      offset += m
  else:
    for s in range(0, groups):
      res.append(
          blocks[s:s+groups+offset:groups] +
          list(range(offset + 1, (offset + 1 + m))))
      offset += m

  return res


def mapNodes(blocks, hosts, offset=0):
  res = {"node" + str((h % hosts)+1): [] for h in range(1, hosts+1)}
  for i, blk in enumerate(blocks):
    res["node" + str((i % hosts)+1)].append(blocks[i])

  return res


MaxBlocks = 256
K = widgets.IntSlider(description="K", min=2, max=MaxBlocks)
M = widgets.IntSlider(description="M", min=1, max=MaxBlocks)
BC = widgets.IntSlider(description="Origina blocks count",
                       min=1, max=10000, value=10)
D = widgets.Dropdown(options=['Horizontal', 'Vertical'],
                     description="EC Coding Direction")
Manifest = widgets.Dropdown(
    options=['Inline', 'Append'], description="Blocks Manifest Layout")
Nodes = widgets.IntSlider(
    description="Nodes", min=K.value + M.value, max=MaxBlocks)

ui = widgets.VBox([K, M, BC, D, Manifest, Nodes])


def updateKRange(*args):
    K.max = MaxBlocks - M.value


def updateMRange(*args):
    M.max = MaxBlocks - K.value


M.observe(updateKRange, 'value')
K.observe(updateMRange, 'value')


def updateNodesRange(*args):
    Nodes.min = K.value + M.value


Nodes.observe(updateNodesRange, 'value')

out = widgets.interactive_output(
    main,
    {
        'K': K,
        'M': M,
        'BC': BC,
        'Dir': D,
        'Man': Manifest,
        'Nodes': Nodes})

display(ui, out)


Output()